In [ ]:
import os
import tqdm
import pandas as pd
import csv
from pandas_ods_reader import read_ods

In [ ]:
warm = ['soil_c','surf_water','soil_temp','flux_soc','soil_water','n_flux','p_flux','temp']

dirs = ['warm_temp_maize_soybean_irrigated', 'warm_temp_maize-soybean_dryland', 'cool_temp_maize_soybean']

path = '../datasets/' + dirs[0]
start_year = 2015
growth_period = 8
os.makedirs(path + '/csv_outs', exist_ok=True)  


# ecosys writes the outputs for canopy and plants variables in files with just a '1' symbol in front of the fluxes outputs
# using this mapper to write the csv with the correct name
plant_mapper = {
'flux_soc' : 'plant_c',
'n_flux' : 'plant_n',
'p_flux': 'plant_p',
'soil_c': 'canopcy_c',
'soil_temp' : 'canopy_temp',
'soil_water': 'plant_stress',
'surf_water': 'photosynthesis',
'temp' : 'plant_growth'
}

In [ ]:
def to_csv(txt_file: str, save_path: str, to_rename: str):
    # Open the input text file and output CSV file
    with open(txt_file, 'r') as in_file, open(save_path + to_rename  + '.csv', 'w', newline='') as out_file:
        # Create a CSV writer object
        writer = csv.writer(out_file)

        # Read each remaining line of the input file
        for line in in_file:
            cols = line.strip().split()
            
            #dirty, stupid hack to get rid of bad data
            if cols[0] != 'DOY':
                cols.remove(cols[0])
                
            # Write the columns to the CSV file
            writer.writerow(cols)

In [ ]:
def turn_to_csv(file_names: list, start_year: int, growth_period: int):
    year = start_year
    itr = 0
    for prefix in range(2):
        year = start_year
        for i in tqdm.tqdm(range(growth_period + 1)):
            for file_name in file_names:
                to_rename = str(year) + file_name
                if prefix == 1:
                    to_rename = str(year) + plant_mapper[file_name]
                possible_file = str(prefix) + str(year) + file_name
                file_path = path + '/outputs/' + possible_file
                if os.path.isfile(file_path):
                    to_csv(file_path, path + '/csv_outs/',to_rename)
                # file missing a few letters due to weird fortran buffers, so substract one from the end
                else: 
                    for i in range(4):
                        file_name = file_name[:-i]
                        possible_file = str(prefix) + str(year) + file_name
                        file_path = path + '/outputs/' + possible_file
                        if os.path.isfile(file_path):
                            to_csv(file_path, path + '/csv_outs/', to_rename)


                itr +=1
            year +=1

        

In [ ]:
turn_to_csv(warm, start_year, growth_period)

100%|███████████████████████████████████████████| 14/14 [00:02<00:00,  6.83it/s]
